In [13]:
import cv2
import numpy as np
import glob
import os

def process_videos(video_paths, output_folder):
    points = []
    
    def select_point(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            points.append((x, y))
            cv2.circle(frame, (x, y), 5, (0, 0, 255), -1)
            cv2.imshow('Select Point', frame)
    
    first_frames = []
    
    # Step 1: Extract first frames and collect points
    for video_path in video_paths:
        cap = cv2.VideoCapture(video_path)
        ret, frame = cap.read()
        if ret:
            first_frames.append((frame, video_path))
            cv2.imshow('Select Point', frame)
            cv2.setMouseCallback('Select Point', select_point)
            cv2.waitKey(0)
        cap.release()
    
    cv2.destroyAllWindows()
    
    # Step 2: Calculate mean point
    if not points:
        print("No points were selected.")
        return
    
    mean_point = np.mean(points, axis=0).astype(int)
    print(f"Mean point: {mean_point}")
    
    # Step 3: Align videos
    os.makedirs(output_folder, exist_ok=True)
    
    for (frame, video_path), point in zip(first_frames, points):
        dx, dy = mean_point[0] - point[0], mean_point[1] - point[1]
        cap = cv2.VideoCapture(video_path)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        output_path = os.path.join(output_folder, os.path.basename(video_path))
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            translation_matrix = np.float32([[1, 0, dx], [0, 1, dy]])
            translated_frame = cv2.warpAffine(frame, translation_matrix, (width, height))
            out.write(translated_frame)
        
        cap.release()
        out.release()
    
    print(f"Aligned videos saved in '{output_folder}'.")

In [5]:
import cv2
import numpy as np
import glob
import os

def process_videos_with_alignment(video_paths, output_folder):
    point_pairs = []  # To store pairs of points for each video

    def select_points(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            param.append((x, y))  # Append the selected point
            cv2.circle(frame, (x, y), 5, (0, 0, 255), -1)
            cv2.imshow('Select Points', frame)
            if len(param) == 2:  # Stop when two points are selected
                cv2.destroyWindow('Select Points')

    first_frames = []

    # Step 1: Extract first frames and collect two points for each video
    for video_path in video_paths:
        cap = cv2.VideoCapture(video_path)
        ret, frame = cap.read()
        if ret:
            first_frames.append((frame, video_path))
            points = []
            cv2.imshow('Select Points', frame)
            cv2.setMouseCallback('Select Points', select_points, points)
            cv2.waitKey(0)
            point_pairs.append(points)
        cap.release()

    cv2.destroyAllWindows()

    # Step 2: Calculate mean points
    if not point_pairs:
        print("No points were selected.")
        return

    mean_points = np.mean(point_pairs, axis=0)
    mean_point1, mean_point2 = mean_points.astype(int)
    print(f"Mean points: {mean_point1}, {mean_point2}")

    # Step 3: Align videos (translate, rotate, resize)
    os.makedirs(output_folder, exist_ok=True)
    mean_vector = mean_point2 - mean_point1
    mean_length = np.linalg.norm(mean_vector)
    mean_angle = np.arctan2(mean_vector[1], mean_vector[0])

    for (frame, video_path), points in zip(first_frames, point_pairs):
        point1, point2 = points
        vector = np.array(point2) - np.array(point1)
        angle = np.arctan2(vector[1], vector[0])
        length = np.linalg.norm(vector)

        scale = mean_length / length
        rotation_angle = np.degrees(mean_angle - angle)

        cap = cv2.VideoCapture(video_path)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        output_path = os.path.join(output_folder, os.path.basename(video_path))
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Translation to align point1 with mean_point1
            dx, dy = mean_point1[0] - point1[0], mean_point1[1] - point1[1]
            translation_matrix = np.float32([[1, 0, dx], [0, 1, dy]])
            frame = cv2.warpAffine(frame, translation_matrix, (width, height))

            # Rotation and scaling around mean_point1
            M = cv2.getRotationMatrix2D((mean_point1[0], mean_point1[1]), rotation_angle, scale)
            frame = cv2.warpAffine(frame, M, (width, height))

            out.write(frame)

        cap.release()
        out.release()

    print(f"Aligned videos saved in '{output_folder}'.")

In [6]:
video_files = glob.glob("C:/Users/dhers/Desktop/prueba/*.mp4")  # Update path to your videos
output_dir = "C:/Users/dhers/Desktop/prueba/aligned_videos"
process_videos_with_alignment(video_files, output_dir)

Mean points: [123  64], [470  63]
Aligned videos saved in 'C:/Users/dhers/Desktop/prueba/aligned_videos'.
